In [60]:
import math
import os
import pandas as pd

import rvms

csv_dir = '../../../csvFiles/'
if csv_dir == '':
    raise ValueError('Specificare un percorso valido!')

def load_csv(path: str) -> pd.DataFrame:
    return pd.read_csv(path, sep='\\s*,\\s*', engine='python')

def confidence_interval(values: list[float], loc: float=0.95) -> float:
    n: int = len(values)
    if n <= 1:
        raise ValueError("list of 2+ elements required")
    mean = sum(values) / n
    std = sum(map(lambda x: (x - mean)**2, values)) / n
    u = 1.0 - 0.5 * (1.0 - loc)
    t = rvms.idfStudent(n - 1, u)
    return t * std / math.sqrt(n - 1)
    

In [62]:
def format_center_name(s: str) -> str:
    return s.replace('_', ' ')

def create_table(
        simulation_type = ('FINITE_SIMULATION', 'IMPROVED_FINITE_SIMULATION'),
        indexes = ('E[Ns]', 'E[Ts]', 'E[Nq]', 'E[Tq]', 'E[s]', 'ρ', 'λ')):
    data: list[list[str]] = []
    # ['Reparto Istruttorie', 'Scoring Automatico', 'Comitato Credito', 'Reparto Liquidazioni']
    for sim in simulation_type:
        parent = os.path.join(csv_dir, sim, 'results')
        for file_name in os.listdir(parent):
            center_name: str = file_name.replace('.csv', '')
            path = os.path.join(parent, file_name)
            df: pd.DataFrame = load_csv(path)
            row: list[str] = [format_center_name(center_name)]
            for i in indexes:
                x = df[i].mean()
                ci = confidence_interval(df[i])
                cell = f'${x:.4f}\\pm{ci:.8f}$'
                row.append(cell)
            data.append(row)
        columns = ['Centro']
        columns.extend(map(lambda i: f'${i}$', indexes))
        table = pd.DataFrame(data, columns=columns)
        table.style.hide(axis="index").to_latex(f'{sim}_output.tex', hrules=True)

In [63]:
create_table(indexes=['E[Ns]', 'E[Ts]', 'E[Nq]', 'E[Tq]'])